In [1]:
# Imports
import os
import bz2
import numpy as np
from PIL import TiffImagePlugin
from IPython.core.display import display
from PIL import Image as Img
import sys
sys.path.append("../")
from algorithms.flow import flow
from data_structures.location_graph import LocationGraph
from utils.dummy_writer import DummyWriter
from utils.image_writer import ImageWriter

In [2]:
def load_image_data() -> TiffImagePlugin.TiffImageFile:
    path = os.path.dirname(os.path.abspath('')) + '/data/'
    data = bz2.decompress(open(path + 'dem.tif.bz2', 'rb').read())
    with open(path + 'dem2.tiff', 'wb') as handle:
        handle.write(data)
    return Img.open(path + 'dem2.tiff')

def load_and_crop_data(crop_size, offset):
    raw_height_array = np.asarray(load_image_data())
    return raw_height_array[offset:offset+crop_size, offset:offset+crop_size]

In [3]:
def normalize_values(i):
    return int((i-1)*255/1193)
def get_colour_function(flows):
    return normalize_values

In [4]:
# Params
size = 500
frequency = 1000
job_name = "small"

def report_settings():
    num_images = int(size**2/frequency)
    print(f"Num images: {num_images}")
    total_size = ((size/1000)**2)*num_images
    if total_size > 1000:
        print(f"Total data size: {total_size/1000.0:.2f} GB")
    else:
        print(f"Total data size: {total_size:.2f} MB")
report_settings()

Num images: 250
Total data size: 62.50 MB


In [5]:
%%time
height_map = load_and_crop_data(size,0).tolist()
print("Creating Graph")
graph = LocationGraph(height_map)

Creating Graph
CPU times: user 2.91 s, sys: 151 ms, total: 3.06 s
Wall time: 3.06 s


In [6]:
%%time
print("Running flow for colour calibration")
nodes_with_flow = flow(graph, DummyWriter())
flows = [n.flow for n in nodes_with_flow.ascending()]
writer = ImageWriter(size, frequency, job_name, get_colour_function(flows))

Running flow for colour calibration
Writer will save images to /home/andrew/personal/river-flow/data/small/
CPU times: user 826 ms, sys: 153 µs, total: 826 ms
Wall time: 825 ms


In [7]:
%%time
print("Running real flow simulation")
flow(graph, writer)

Running real flow simulation
Processed 2% of 250000 images.
Processed 4% of 250000 images.
Processed 6% of 250000 images.
Processed 8% of 250000 images.
Processed 10% of 250000 images.
Processed 12% of 250000 images.
Processed 14% of 250000 images.
Processed 16% of 250000 images.
Processed 18% of 250000 images.
Processed 20% of 250000 images.
Processed 22% of 250000 images.
Processed 24% of 250000 images.
Processed 26% of 250000 images.
Processed 28% of 250000 images.
Processed 30% of 250000 images.
Processed 32% of 250000 images.
Processed 34% of 250000 images.
Processed 36% of 250000 images.
Processed 38% of 250000 images.
Processed 40% of 250000 images.
Processed 42% of 250000 images.
Processed 44% of 250000 images.
Processed 46% of 250000 images.
Processed 48% of 250000 images.
Processed 50% of 250000 images.
Processed 52% of 250000 images.
Processed 54% of 250000 images.
Processed 56% of 250000 images.
Processed 58% of 250000 images.
Processed 60% of 250000 images.
Processed 62% o